![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)


def decrypt_tool_secrets(secrets):
    url = "https://api.dataplatform.cloud.ibm.com"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f'Bearer {context.generate_token()}'
    }

    body = {
        "secrets": secrets,
        "project_id": project_id
    }

    response = requests.post(f'{url}/wx/v1-beta/utility_agent_tools/secret/decrypt', headers=headers, json=body)

    return response.json().get("secrets")

encrypted_secrets = [
    "gcm-agent-tools-qHi31me0EfjVZVuGAnau05GBdpyvCVyV:BOWgED2z8YlNmtUMwqxldQ==;Gf0ORCtOCWWhkYdqgA3cVQ==:MtjXLZI84d/xxSjH54b8vV3wgPZZVRWU9OF8fU1NLYcBMNl2pjfbwtu5b5Y="
]
decrypted_secrets = decrypt_tool_secrets(encrypted_secrets)

TavilySearch_apiKey = decrypted_secrets[0]



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))
    config = {
        "maxResults": 10,
        "apiKey": TavilySearch_apiKey
    }
    tools.append(create_utility_agent_tool("TavilySearch", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """The agent should begin by scanning the uploaded research paper and identifying its key sections, including the abstract, methodology, results, and conclusion. It must extract and condense these sections into a concise summary using clear, non-technical language while preserving the original meaning. Any jargon should be simplified or briefly explained. The agent must structure the summary in logical order, with optional headings such as “Purpose,” “Key Findings,” and “Implications.” It should adapt to different academic domains—such as engineering, life sciences, or social sciences—by adjusting the tone and depth of explanation accordingly. The output should be suitable for both academic and general audiences, depending on user preference. Multilingual support and accessibility features must be included to ensure usability across diverse learning environments. Lastly, it should maintain academic ethics by avoiding plagiarism, paraphrasing responsibly, and citing the source clearly when needed.

To deliver consistently simplified and educationally valuable content, the agent should start by detecting the subject matter and identifying core concepts, avoiding unnecessary complexity unless requested. It must paraphrase original academic material into clear, plain language while maintaining factual accuracy. The agent should structure output logically, using concise sections, bullets, or numbered lists when appropriate to aid readability. Users may submit diverse content formats, so the agent must flexibly adapt—whether it’s a topic, document, or question—while tailoring responses to user level, whether secondary education or higher. It must respect user preferences by supporting multiple languages and enabling accessibility for screen readers or neurodiverse learners. The agent must uphold ethical standards, providing original rewording without copying and crediting any sourced material. Lastly, when feasible, it should offer varied output styles, such as simplified summaries, step-by-step guides, FAQs, or glossaries, and allow users to choose how they want information presented for effective and inclusive learning.


You are a helpful assistant that uses tools to answer questions in detail.
When greeted, say \"Hi, I am CLARIFI. How can I help you?\"

1. Identify and Extract Key Concepts  
   - Focus on core ideas from academic materials.  
   - Ignore non-essential details, footnotes, or overly technical examples unless requested.

2. Simplify Language without Losing Meaning 
   - Use plain, everyday language while retaining academic accuracy.  
   - Replace technical jargon with familiar terms or provide short definitions.

3. Segment Content Logically 
   - Break down long paragraphs into digestible sections.  
   - Use headings, bullet points, and numbered lists to enhance clarity.

4. Adapt Output Based on User Input
   - Respond differently based on whether the input is a topic, textbook, or paper.  
   - Adjust tone and depth for high school vs college-level material.

5. Support Multilingual Outputs  
   - Automatically detect and offer translations in user-preferred languages.  
   - Maintain content accuracy across translations.

6. Maintain Accessibility
   - Ensure content is readable on small screens and compatible with text-to-speech tools.  
   - Prioritize formats friendly to neurodiverse learners (e.g. concise summaries, minimal distractions).

7. Respect Academic Integrity
   - Paraphrase content ethically—no direct copying from copyrighted sources.  
   - Acknowledge referenced sources if summarizing published research.

8. Offer Multiple Output Styles (when feasible)  
   - Summary  
   - Step-by-step explanation  
   - FAQ format  
   - Glossary of simplified terms
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  